In [27]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [28]:
# 读取全部.csv文件并直接堆叠
all_df = []
all_data_path = "data/"
all_df_names = os.listdir(all_data_path)
for name in all_df_names:
    df = pd.read_csv(os.path.join(all_data_path, name))
    df['date'] = pd.to_datetime(name[5:13])
    df['timestamp'] = pd.to_timedelta(df['timestamp'])
    all_df.append(df)

df = pd.concat(all_df)

# 转换时间格式
df['datetime'] = pd.to_datetime(df['timestamp']+df['date'])

In [29]:
# 按照交易时间段分组
def to_period(t):
    if pd.to_timedelta('09:00:00') <= t <= pd.to_timedelta('10:15:00'):
        return 1
    elif pd.to_timedelta('10:30:00') <= t <= pd.to_timedelta('11:30:00'):
        return 2
    elif pd.to_timedelta('13:30:00') <= t <= pd.to_timedelta('15:00:00'):
        return 3
    elif pd.to_timedelta('21:00:00') <= t <= pd.to_timedelta('23:00:00'):
        return 4
    return pd.NA
    
df['period'] = df['timestamp'].apply(to_period)
df = df.dropna(subset=['period','last_price']).reset_index(drop=True)

In [30]:
# 样例
df.sample(5)

,timestamp,last_price,volume,turnover,bid_price1,bid_price2,bid_price3,bid_price4,bid_price5,ask_price1,...,bid_volume4,bid_volume5,ask_volume1,ask_volume2,ask_volume3,ask_volume4,ask_volume5,date,datetime,period
183496,0 days 22:45:15.435547,13515.0,57635,781072025,13515.0,13510.0,13505.0,13500.0,13495.0,13520.0,...,1116.0,401.0,367.0,708.0,393.0,333.0,260.0,2025-09-25,2025-09-25 22:45:15.435547,4
9205,0 days 09:43:11.718789,13575.0,117478,1601897090,13570.0,13565.0,13560.0,13555.0,13550.0,13575.0,...,761.0,1266.0,302.0,1278.0,629.0,551.0,303.0,2025-09-22,2025-09-22 09:43:11.718789,1
69956,0 days 13:35:41.699407,13520.0,5336,72180905,13520.0,13515.0,13510.0,13505.0,13500.0,13525.0,...,2295.0,1719.0,504.0,467.0,484.0,577.0,258.0,2025-09-23,2025-09-23 13:35:41.699407,3
224953,0 days 22:21:01.635292,13485.0,46499,627219525,13485.0,13480.0,13475.0,13470.0,13465.0,13490.0,...,641.0,307.0,639.0,1149.0,660.0,597.0,350.0,2025-09-26,2025-09-26 22:21:01.635292,4
61912,0 days 10:33:40.758022,13525.0,44913,608561755,13525.0,13520.0,13515.0,13510.0,13505.0,13530.0,...,3220.0,2384.0,1243.0,442.0,532.0,320.0,197.0,2025-09-23,2025-09-23 10:33:40.758022,2


In [ ]:
### 这一部分是进一步数据清洗和检验，没有标准流程，下列注释是最后保留的代码，其中需要特别关注的是高频数据中断的问题

# df = df.drop_duplicates(subset=['datetime'], keep='first') # 检查是否有重复时间点：无
# all(df == df.sort_values('datetime')) # 检查时间是否按顺序排列：是
# (df.iloc[:,4:24]==0).to_numpy().any() # 检查是否存在无价格情况：无

### 检查是否有长时间数据缺失：

# df['gap'] = (df['datetime'].diff() > pd.to_timedelta('00:00:05'))
# time_gaps = df[df['gap']] 
# sum(df['timestamp'].diff()<pd.to_timedelta('00:00:00.25'))
# time_gaps.shape[0] 

### 连续15秒没有记录0次/连续10秒没有记录7次/连续5秒没有记录249次
### 约70%的数据间隔在0.25s左右，约15%的数据间隔在0.5s左右，约10%的数据间隔在0.75s和0.1s左右
### 这表明原始数据是基本固定时间抓取一次期货数据的，并且存在一定波动

In [31]:
# 特征工程和采样

# freq: 采样频率；gap_max：当数据中断超过该值时进行片段切分；window_look_before：计算因子时向过去看的窗口时长
def rv_sample(df_sample, freq=pd.Timedelta(seconds=1), gap_max=pd.Timedelta(seconds=5), window_look_before=pd.Timedelta(seconds=60)):
    samples = []

    # 过去看窗口时长秒数
    second_look_before = window_look_before.total_seconds()

    # 计算原始数据间隔并分段
    df_sample['gap'] = df_sample['datetime'].diff() > gap_max
    df_sample['segment'] = df_sample['gap'].cumsum()

    # 预先计算部分因子所需变量
    df_sample['log_last_price'] = np.log(df_sample['last_price'])
    # df_sample['log_last_price_squared_change'] = np.square(df_sample['log_last_price'].diff())
    df_sample_bid_price = df_sample[[f'bid_price{i}' for i in range(1, 6)]]
    df_sample_ask_price = df_sample[[f'ask_price{i}' for i in range(1, 6)]]
    df_sample_bid_volume = df_sample[[f'bid_volume{i}' for i in range(1, 6)]]
    df_sample_ask_volume = df_sample[[f'ask_volume{i}' for i in range(1, 6)]]
    df_sample['weighted_bid_price'] = (df_sample_bid_price.to_numpy() * df_sample_bid_volume.to_numpy()).sum(axis=1) / (df_sample_bid_volume.sum(axis=1)+1e-5)
    df_sample['weighted_ask_price'] = (df_sample_ask_price.to_numpy() * df_sample_ask_volume.to_numpy()).sum(axis=1) / (df_sample_ask_volume.sum(axis=1)+1e-5)
    df_sample['bid_volume_sum'] = df_sample_bid_volume.sum(axis=1)
    df_sample['ask_volume_sum'] = df_sample_ask_volume.sum(axis=1)

    # 对每个片段，采样开始后60s至结束前60s
    for seg, df_seg in tqdm(df_sample.groupby('segment')):

        seg_sample_start = (df_seg['datetime'].iloc[0] + pd.Timedelta(seconds=61)).floor('1s')
        seg_sample_end = (df_seg['datetime'].iloc[-1] - pd.Timedelta(seconds=60)).floor('1s')

        if seg_sample_start>seg_sample_end:
            continue

        seg_period = df_seg['period'].iloc[0]
        
        seg_sample_time = pd.date_range(start=seg_sample_start, end=seg_sample_end, freq=freq)

        df_seg = df_seg.set_index('datetime')

        # 对每个采样点，获取过去窗口和未来1分钟的所有数据
        for t in seg_sample_time:
            df_seg_before = df_seg.loc[(t-window_look_before):t]
            df_seg_after = df_seg.loc[(df_seg.index > t)&(df_seg.index <= t+pd.Timedelta(seconds=60))]

            # 过去窗口时间权重
            df_seg_before_time_weights = df_seg_before.index.to_series().diff().dt.total_seconds().fillna(0)
            df_seg_before_length = (df_seg_before.index[-1] - df_seg_before.index[0]).total_seconds()
            
            # 未来1分钟波动率
            second_var = ((df_seg_after['log_last_price'].diff())**2).sum()
            rv = np.log(second_var + 1e-5) / 60
            
            # 成交价因子
            # 1.1.过去窗口的成交价波动率
            past_log_return = df_seg_before['log_last_price'].diff()
            past_second_var = (past_log_return**2).sum()
            past_rv = np.log(past_second_var + 1e-5) / second_look_before
            # 1.2.过去窗口的成交价范围
            past_price_range = df_seg_before['log_last_price'].max() - df_seg_before['log_last_price'].min()
            # 1.3.过去窗口的累积收益率
            past_ac_log_return = df_seg_before['log_last_price'].iloc[-1] - df_seg_before['log_last_price'].iloc[0]
            # 1.4.过去窗口的正收益刻占比
            past_up_ratio = ((past_log_return>0) * df_seg_before_time_weights).sum() / df_seg_before_length
            
            # 买卖因子
            # 2.1.过去窗口内买卖价加权平均
            past_weighted_bid_ask_price_sum = df_seg_before['weighted_bid_price'] + df_seg_before['weighted_ask_price']
            past_weighted_bid_ask_price = (past_weighted_bid_ask_price_sum * df_seg_before_time_weights).sum() / df_seg_before_length
            # 2.2.过去窗口内买卖加权中价变化范围
            past_weighted_mid_bid_ask_price_range = past_weighted_bid_ask_price_sum.max() - past_weighted_bid_ask_price_sum.min()
            # 2.3.过去窗口内买卖价差加权平均
            past_weighted_bid_ask_price_diff_cut = df_seg_before['weighted_bid_price'] - df_seg_before['weighted_ask_price']
            past_weighted_bid_ask_price_diff = (past_weighted_bid_ask_price_diff_cut * (
                df_seg_before_time_weights)).sum() / df_seg_before_length
            # 4.过去窗口内买卖数量不平衡加权平均
            past_weighted_bid_ask_volume_diff_cut = ((df_seg_before['bid_volume_sum'] - df_seg_before['ask_volume_sum']) / (
                (df_seg_before['bid_volume_sum'] + df_seg_before['ask_volume_sum']))).abs()
            past_weighted_bid_ask_volume_diff = (past_weighted_bid_ask_volume_diff_cut * 
                    df_seg_before_time_weights).sum() / df_seg_before_length

            # 总量因子
            # 3.1.过去窗口的总成交量
            past_total_volume = df_seg_before['volume'].iloc[-1] - df_seg_before['volume'].iloc[0]
            # 3.2.过去窗口的总成交价格
            past_total_value = df_seg_before['turnover'].iloc[-1] - df_seg_before['turnover'].iloc[0]
            # 3.3.过去窗口带驱动方向的总成交量
            past_price_sign = np.sign(df_seg_before['last_price'].diff())
            past_price_sign = past_price_sign.replace(0, np.nan).ffill()
            past_signed_volume = (past_price_sign * df_seg_before['volume'].diff()).sum()
            # 3.4.过去窗口带驱动方向的成交量波动率
            past_signed_volume_var = (past_price_sign * df_seg_before['volume'].diff()).std()

            # 采样点前一刻数据点的类似信息
            # 1.4
            past_tick_log_return = df_seg_before['log_last_price'].iloc[-1] - df_seg_before['log_last_price'].iloc[-2]
            # 2.1,2.3,2.4
            past_tick_weighted_bid_ask_price = past_weighted_bid_ask_price_sum.iloc[-1]
            past_tick_weighted_bid_ask_price_diff = past_weighted_bid_ask_price_diff_cut.iloc[-1]
            past_tick_weighted_bid_ask_volume_diff = past_weighted_bid_ask_volume_diff_cut.iloc[-1]
            # 3.1,3.2
            past_tick_volume = df_seg_before['volume'].iloc[-1] - df_seg_before['volume'].iloc[-2]
            past_tick_value = df_seg_before['turnover'].iloc[-1] - df_seg_before['turnover'].iloc[-2]
            past_tick_signed_volume = past_price_sign.iloc[-1] * past_tick_volume

            # 添加采样结果
            samples.append({
                'segment': seg,
                'period': seg_period,
                't': t,
                'rv': rv,
                
                
                # 1
                'past_rv': past_rv,
                'past_price_range': past_price_range,
                'past_ac_log_return': past_ac_log_return,
                'past_up_ratio': past_up_ratio,
                
                # 2
                'past_weighted_bid_ask_price': past_weighted_bid_ask_price,
                'past_weighted_mid_bid_ask_price_range': past_weighted_mid_bid_ask_price_range,
                'past_weighted_bid_ask_price_diff': past_weighted_bid_ask_price_diff,
                'past_weighted_bid_ask_volume_diff': past_weighted_bid_ask_volume_diff,

                # 3
                'past_total_volume': past_total_volume,
                'past_total_value': past_total_value,
                'past_signed_volume': past_signed_volume,
                'past_signed_volume_var': past_signed_volume_var,

                # 时刻特征
                'past_tick_log_return': past_tick_log_return,
                'past_tick_weighted_bid_ask_price': past_tick_weighted_bid_ask_price,
                'past_tick_weighted_bid_ask_price_diff': past_tick_weighted_bid_ask_price_diff,
                'past_tick_weighted_bid_ask_volume_diff': past_tick_weighted_bid_ask_volume_diff,
                'past_tick_volume': past_tick_volume,
                'past_tick_value': past_tick_value,
                'past_tick_signed_volume': past_tick_signed_volume,

            })
            
    return pd.DataFrame(samples)

In [32]:
result = rv_sample(df)

100%|██████████| 269/269 [02:43<00:00,  1.64it/s]


In [33]:
result.to_csv('samples.csv')